In [32]:
import logging

from langchain_community.document_loaders import TextLoader
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA

In [2]:
loaders = [TextLoader('./data/How_to_invest_money.txt', encoding='utf-8')]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [4]:
vectorstore = Chroma(collection_name='split_parents', embedding_function=OpenAIEmbeddings())
store = InMemoryStore()

In [5]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)
retriever.add_documents(docs)
print(len(list(store.yield_keys())))

219


In [20]:
query = 'What are the types of investments?'
retrieved_docs = retriever.invoke(query)
print(retrieved_docs[0].page_content)

There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.


In [11]:
sub_docs = vectorstore.similarity_search(query)
print(sub_docs[0].page_content)

forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.


In [22]:
logging.basicConfig()
logging.getLogger('langchain_classic.retrievers.multi_query').setLevel(logging.INFO)

In [23]:
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = recursive_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

In [26]:
model = ChatOpenAI(model='gpt-5-nano', temperature=0.2)

In [27]:
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=model
)

In [28]:
question = '주식 투자를 처음 시작하려면 어떻게 해야 하나요?'
unique_docs = retriever.invoke(question)
print(f'{len(unique_docs)}개의 문서가 검색되었습니다.')

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작하려면 어떤 순서로 준비하고 어떤 기초지식을 먼저 익히면 좋나요?', '초보자로서 주식 투자에 입문할 때 알아두면 좋은 기본 원칙과 필요한 준비물은 무엇인가요?', '주식 초보가 초기 투자를 시작하기 전에 피해야 할 실수와 권장되는 초기 투자 전략은 무엇인지 알고 싶습니다.']


4개의 문서가 검색되었습니다.


In [31]:
print(unique_docs[0].page_content)

After learning how to judge the value of every form of investment, a man
may still be unsuccessful in the investment of money unless he acquires
also a firm grasp upon the general principles which control the price
movements of securities. By this it is not meant that a man needs to
have an intimate knowledge of technical market conditions whereby to
estimate temporary fluctations of minor importance, but rather that he
should have clearly in mind the causes which operate to produce the
larger swings of prices. If an investor acquires such a knowledge, he is
enabled to take advantage of large price movements in such a way as
materially to increase his income, and, at the same time, avoid
carrying upon his books securities which may have cost much more than
their current market quotations. If he can recognize the indications
which point to the beginning of a pronounced upward swing in securities,
and if he can equally recognize the signs which indicate that the


In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True
)

In [34]:
result = qa_chain.invoke({'query':question})
print(result['result'])
for doc in result['source_documents']:
    print(doc.page_content)

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작하는 사람에게 필요한 기본 학습 순서와 실제 시작 방법은 무엇인가요?', '초보 투자자를 위한 주식 투자 입문 로드맵과 첫 포트폴리오 구성 방법은 어떻게 되나요?', '주식 초보가 피해야 할 일반적인 실수와 함께 위험 관리와 기본 원칙을 반영한 구체적 시작 계획은 어떻게 세우면 되나요?']


다음은 George Garr Henry의 읽기에서 제시하는 기본 원칙에 근거한 시작 방법입니다.

- 주식과 채권의 차이를 먼저 이해한다
  - 같은 투자라도 성격과 위험 관리 방식이 다르다는 기본 개념을 분명히 한다.

- 큰 가격 변동의 원인을 이해하는 데 집중한다
  - 단순한 단기 변동이나 기술적 신호에 매달리기보다, 주가의 장기적 움직임을 좌우하는 요인을 파악하려고 한다.

- 위험 분산과 실질적 필요에 맞춘 선택 원칙을 적용한다
  - 두 가지 핵심 원칙: 위험을 분산시키는 것과 본인의 실제 필요(목표, 기간, 유동성 등)에 맞춰 증권을 고르는 것.

- 지나친 수익 약속에 현혹되기 못지 않게 안전성도 고려한다
  - 투자에서 고수익을 약속하는 것일수록 위험이 크며, 안전성을 희생하는 방식은 피하는 것이 좋다.

- 시장의 기술적 조건에 지나치게 의존하기보다는 기본 원칙을 바탕으로 배운다
  - 깊은 기술 분석 지식이 없어도 큰 가격 움직임의 원인을 이해하는 것이 성공의 중요한 부분이라고 본다.

- 차분하게 자신만의 투자 원칙을 세운 뒤 천천히 배워나간다
  - 초기에는 위험 분산이 가능한 간단한 포트폴리오 구성부터 시작해 점차 개선해 나가는 것이 바람직하다고 제시된다.

필요하면 이 원칙들을 바탕으로 본인의 목적과 위험 허용도에 맞춘 구체적 계획(목표 수익, 투자 기간, 필요 현금 여부 등)을 세워 보세요. 원문은 기본적으로 “주식은 채권과 다르고, 큰 가격 움직임의 원인을 이해하며, 위험 분산과 실질 필요에 맞춘 선택이 중요하다”는 점을 강조합니다.
G. G. H.




HOW TO INVEST MONEY




I

GENERAL PRINCIPLES OF INVESTMENT
After learning how to judge the value of every form of investment, a man
may still be unsuccessful in the investment of money unless he acquires
also a